
# 📊 Projeto de Segmentação de Clientes — E-commerce (RFM, PCA, Clustering, Business)

Neste projeto, utilizei o dataset **Ecommerce**, que contém informações como *InvoiceNo, StockCode, Country*, entre outras variáveis transacionais, com o objetivo de segmentar clientes e extrair insights de negócio.

O fluxo do projeto inclui:

- **Data Cleaning**: remoção de valores inválidos, tratamento de dados ausentes e ajustes necessários para análise.
- **Feature Engineering**: criação das métricas de **RFM (Recency, Frequency, Monetary)**.
- **Análise Exploratória (EDA)** do dataset original e das variáveis RFM.
- **Análise estatística** descritiva tanto do dataset quanto dos clusters gerados.
- Uso de **winsorization** para lidar com caudas longas nas visualizações.
- **Tratamento de outliers** utilizando percentis e o algoritmo **DBSCAN**.
- Aplicação de **PCA** para redução de dimensionalidade e visualização em 2D.
- Treinamento do modelo **K-Means** para segmentação de clientes.
- Visualizações dos clusters no espaço original e no espaço reduzido pelo PCA.
- Análise detalhada dos clusters obtidos, incluindo:
  - identificação de clientes que cancelaram,
  - criação de *naming* dos grupos (**VIP, Regulares e Inativos**),
  - propostas de estratégias de retenção e engajamento para cada segmento.

As principais bibliotecas utilizadas foram: **pandas**, **numpy**, **matplotlib**, **seaborn** e módulos do **scikit-learn**, incluindo **PCA**, **DBSCAN** e **KMeans**.

O objetivo final é demonstrar um pipeline completo de análise e segmentação de clientes, combinando estatística, machine learning e interpretação de negócio.

## 📁Colunas do dataset :



| Coluna        | Descrição |
|--------------|-----------|
| **InvoiceNo** | Identificador único da fatura (nota fiscal). Valores que começam com **"C"** indicam **cancelamentos ou devoluções**. |
| **StockCode** | Código único do produto no sistema da empresa. |
| **Description** | Descrição textual do produto vendido. |
| **Quantity** | Quantidade de unidades do produto na transação.  |
| **InvoiceDate** | Data e hora em que a transação foi realizada. |
| **UnitPrice** | Preço unitário do produto.|
| **CustomerID** | Identificador único do cliente. Pode conter valores ausentes para clientes não cadastrados. |
| **Country** | País onde o cliente está localizado. |

## 📌 Contexto e Propósito do Projeto

Em cenários de e-commerce, compreender o comportamento dos clientes é essencial para tomar decisões estratégicas baseadas em dados. Bases transacionais costumam conter padrões ocultos de consumo que, quando bem explorados, permitem segmentar clientes e direcionar ações de negócio de forma mais eficiente.

O propósito deste projeto é **identificar grupos de clientes com comportamentos semelhantes**, utilizando técnicas de análise exploratória, estatística e aprendizado não supervisionado. A partir dessa segmentação, torna-se possível:

- **Separar clientes em clusters distintos**, com base em padrões de recência, frequência e valor gasto (RFM);
- **Analisar o perfil de cada grupo**, entendendo nível de engajamento e valor gerado;
- **Extrair insights acionáveis** para apoiar decisões de marketing e retenção;
- **Criar estratégias específicas para cada cluster**, em vez de abordagens genéricas;
- **Melhorar a retenção de clientes ativos**, reduzindo churn;
- **Reativar clientes inativos**, por meio de campanhas direcionadas;
- **Estimular a migração de clientes regulares para o grupo VIP**, aumentando frequência e ticket médio;
- Apoiar estratégias para **atração de novos clientes**, usando o perfil dos melhores consumidores como referência.

Assim, o projeto demonstra como técnicas de **clustering e análise estatística** podem ser usadas para transformar dados brutos de e-commerce em conhecimento estratégico, com impacto direto em retenção, engajamento e geração de valor para o negócio.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw.csv', encoding='latin1')
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


Podemos notar através da função describe que temos valores **negativos** em Quantity (Quantidade) e UnitPrice (Preço unitário), isso quer dizer que em Quantity há cancelamentos e em UnitPrice há erros já que preço não pode ser negativo.

Além disso, nota-se que os valores máximos de Quantity e UnitPrice estão **muito acima do percentil 75%** , evidenciando uma distribuição **altamente assimétrica**. Enquanto 75% dos registros apresentam Quantity ≤ 10 e UnitPrice ≤ 4,13, existem observações com valores extremamente elevados, que se distanciam significativamente da média.

Apesar da média estar próxima do percentil 75%, indicando uma **grande concentração de valores baixos**, a presença desses valores extremos exerce forte influência nas estatísticas descritivas, caracterizando a **existência de outliers relevantes**.

Estes outliers **devem** ser tratados pois eles influenciam tanto na predição do modelo quanto na visualização gráfica (A escala fica muito grande, **cauda longa**)

In [3]:
df.describe(include='object').T

,count,unique,top,freq
InvoiceNo,541909,25900,573585,1114
StockCode,541909,4070,85123A,2313
Description,540455,4223,WHITE HANGING HEART T-LIGHT HOLDER,2369
InvoiceDate,541909,23260,10/31/2011 14:41,1114
Country,541909,38,United Kingdom,495478


Através deste describe conseguimos descobrir que o **produto mais vendido foi o WHITE HANGING HEART T-LIGHT HOLDER**,  dentre os 38 países **United Kingdom (Reino unido) foi o país que mais foram feito vendas**, o cliente de **InvoiceNo 573585 foi o cliente que mais comprou de uma só vez levando 1114 itens.**

In [4]:
df.sample(7, random_state=1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
94801,C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.0,United Kingdom
210111,555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,NaN,United Kingdom
455946,575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.0,United Kingdom
403542,571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.0,United Kingdom
471951,576657,22556,PLASTERS IN TIN CIRCUS PARADE,12,11/16/2011 11:03,1.65,12720.0,Germany
380570,569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.0,United Kingdom
384867,570185,21090,WET/MOULDY,-192,10/7/2011 14:56,0.00,NaN,United Kingdom


Neste sample conseguimos identificar um registro (Primeiro da lista) que possui um C na frente do InvoiceNo, isso representa um **cancelamento**, iremos **separar os cancelamentos**, **colocar em outro dataframe para fazer análises posteriores** e iremos **excluir as compras canceladas deste dataset**.
Podemos perceber também que no último e no primeiro registro temos valores negativos em Quantity. No primeiro registro está correto ser negativo já que ocorreu um cancelamento indicado pelo C no InvoiceNo, já no último registro do sample o InvoiceNo não indica cancelamento mas Quantity está negativo, ou seja, este é um **registro com erro**, precisamos excluir, **iremos excluir todos os valores negativos em Quantity e Tudo que começa com C em InvoiceNo**.

### Verificando cancelamentos e guardando no dataframe df_cancelamentos :

In [5]:
df_cancelamentos = df[df['InvoiceNo'].str.startswith('C')]
df_cancelamentos.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom


Podemos perceber agora que temos registros especiais em StockCode, como D em Discount, como iremos fazer segmentação e análise de retenção para clientes podemos excluir também estes registros especiais do dataset.

### Excluindo do dataframe os registros de cancelamento ou devolução :

In [6]:
df = df[~df['InvoiceNo'].str.startswith('C')]

### Verificando os registros especiais :

In [7]:
reg_especiais = df['StockCode'].str.match(r'^[A-Za-z]')

df[reg_especiais]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
45,536370,POST,POSTAGE,3,12/1/2010 8:45,18.00,12583.0,France
386,536403,POST,POSTAGE,1,12/1/2010 11:27,15.00,12791.0,Netherlands
1123,536527,POST,POSTAGE,1,12/1/2010 13:04,18.00,12662.0,Germany
1423,536540,C2,CARRIAGE,1,12/1/2010 14:05,50.00,14911.0,EIRE
1814,536544,DOT,DOTCOM POSTAGE,1,12/1/2010 14:32,569.77,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541216,581494,POST,POSTAGE,2,12/9/2011 10:13,18.00,12518.0,Germany
541540,581498,DOT,DOTCOM POSTAGE,1,12/9/2011 10:26,1714.17,NaN,United Kingdom
541730,581570,POST,POSTAGE,1,12/9/2011 11:59,18.00,12662.0,Germany
541767,581574,POST,POSTAGE,2,12/9/2011 12:09,18.00,12526.0,Germany


### retirando - os do dataframe :

In [8]:
df = df[df['StockCode'].str.match(r'^\d')]

### Verificando dados faltantes :

In [9]:
print(df.info())

print('\n Dados faltantes: \n\n',df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 530210 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    530210 non-null  object 
 1   StockCode    530210 non-null  object 
 2   Description  528771 non-null  object 
 3   Quantity     530210 non-null  int64  
 4   InvoiceDate  530210 non-null  object 
 5   UnitPrice    530210 non-null  float64
 6   CustomerID   396370 non-null  float64
 7   Country      530210 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.4+ MB
None

 Dados faltantes: 

 InvoiceNo           0
StockCode           0
Description      1439
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133840
Country             0
dtype: int64


Foi verificado que possuimos **dados faltantes** em **Description (1439 registros)**  e em **CustomerID (133840)**. Nestes casos não podemos imputar nada, CustomerID é um valor aleatório e Description é um produto.
Verificando os tipos de dados temos que nos atentar a colocar o InvoiceDate para formato de data (datetime), irei fazer isso no código abaixo.

In [10]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

### Verificando os dados faltantes :

In [ ]:
df_missing = df[df['CustomerID'].isnull() | df['Description'].isnull()]
df_missing

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541535,581498,85049e,SCANDINAVIAN REDS RIBBONS,4,2011-12-09 10:26:00,3.29,NaN,United Kingdom
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom


É possível perceber que, exceto no primeiro registro que só tem um InvoiceNo único, **todos os outros registros com customerID NAN vem de um mesmo InvoiceNo, ou seja, de uma nota fiscal**. Isso me fez perguntar se teve algum registro onde InvoiceNo combina e existe um CustomerID não nulo (Pode ter ocorrido um erro), se isso ocorre podemos preencher os dados faltantes .

### Verificando matchs :

In [12]:
missing_before = df['CustomerID'].isna().sum()
print(f'CustomerID faltantes ANTES: {missing_before}')


match_cols = [
    'InvoiceDate',
    'StockCode',
    'Quantity',
    'UnitPrice',
    'Country'
]

reference_valid = (
    df.dropna(subset=['CustomerID'])
      .groupby(match_cols)['CustomerID']
      .nunique()
      .reset_index()
)

reference_valid = reference_valid[reference_valid['CustomerID'] == 1]

reference = (
    df.dropna(subset=['CustomerID'])
      .merge(reference_valid[match_cols], on=match_cols, how='inner')
      [match_cols + ['CustomerID']]
      .drop_duplicates()
)

df = df.merge(
    reference,
    on=match_cols,
    how='left',
    suffixes=('', '_filled')
)

df['CustomerID'] = df['CustomerID'].fillna(df['CustomerID_filled'])
df.drop(columns='CustomerID_filled', inplace=True)

missing_after = df['CustomerID'].isna().sum()
print(f'CustomerID faltantes DEPOIS: {missing_after}')
print(f'Recuperados: {missing_before - missing_after}')


CustomerID faltantes ANTES: 133840
CustomerID faltantes DEPOIS: 133840
Recuperados: 0


Não foi possível achar nenhum match, ou seja, **os registros com customerID nulos são podem ser recuperados** , só nos resta excluir do dataset.

### Excluindo valores nulos :

In [13]:
df = df.dropna(subset=['CustomerID', 'Description'])
df.shape

(396370, 8)

In [14]:
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Verificando se ha registros onde quatidade é negtivo :

In [15]:
df[df['Quantity'] <= 0 ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


**Não há registros com quantidade negativa**

### Verificando se há UnitPrice negativo

In [16]:
df[df['UnitPrice'] <= 0 ].shape

(33, 8)

**33 registros onde UnitPrice = 0, precisamos apagar** pois não é possível o preço ser negativo.

In [17]:
df.duplicated().sum()
df_duplicates = df[df.duplicated(keep=False)]
df_duplicates

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
474,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
478,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
483,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
506,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
510,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
...,...,...,...,...,...,...,...,...
529982,581538,22068,BLACK PIRATE TREASURE CHEST,1,2011-12-09 11:34:00,0.39,14446.0,United Kingdom
529996,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
529999,581538,22992,REVOLVER WOODEN RULER,1,2011-12-09 11:34:00,1.95,14446.0,United Kingdom
530006,581538,22694,WICKER STAR,1,2011-12-09 11:34:00,2.10,14446.0,United Kingdom


Estes registros acima são registros onde há **duplicados**, ou seja, estão iguais a outros registros, e portanto devem ser **excluídos**.

In [18]:
df = df.drop_duplicates()
df.shape

(391183, 8)

## Feature engineering :

### Criando a coluna Revenue (Lucro) :

In [19]:
df['Revenue'] = df['Quantity'] * df['UnitPrice']
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
530205,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
530206,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
530207,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
530208,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [20]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,391183.0,13.179665,1.0,2.0,6.0,12.0,80995.0,181.907403
InvoiceDate,391183,2011-07-10 19:37:28.017628416,2010-12-01 08:26:00,2011-04-07 11:16:00,2011-07-31 12:05:00,2011-10-20 12:57:00,2011-12-09 12:50:00,NaN
UnitPrice,391183.0,2.87413,0.0,1.25,1.95,3.75,649.5,4.284639
CustomerID,391183.0,15295.083503,12346.0,13969.0,15158.0,16794.0,18287.0,1710.359579
Revenue,391183.0,22.335397,0.0,4.95,11.9,19.8,168469.6,310.919394


In [21]:
df.to_csv("../data/cleaned.csv", index=False)

Ainda temos **Outliers** e iremos tratar posteriormente